In [20]:
import pandas as pd
import os
from tqdm import tqdm
from PIL import Image

In [21]:
path='data/baselinetest_pred/labels/'
img_path='data/test_frames/'
out='submission.csv'

In [22]:
def rm_overlap(df,iou_threshold=0.8):
    for index1, row1 in df.iterrows():
        bb1,bb2={},{}
        bb1['x1'],bb1['y1']=row1['bb_left'],row1['bb_top']
        bb1['x2'],bb1['y2']=row1['bb_left']+row1['width'],row1['bb_top']+row1['height']
        for index2, row2 in df.iterrows():
            if index2==index1 or row1['class']!= row2['class']: continue
            bb2['x1'],bb2['y1']=row2['bb_left'],row2['bb_top']
            bb2['x2'],bb2['y2']=row2['bb_left']+row2['width'],row2['bb_top']+row2['height']
            if get_iou(bb1,bb2)>iou_threshold:
                df=df.drop(index2)
    return df
            
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : dict
        Keys: {'x1', 'x2', 'y1', 'y2'}
        The (x, y) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """
    assert bb1['x1'] < bb1['x2']
    assert bb1['y1'] < bb1['y2']
    assert bb2['x1'] < bb2['x2']
    assert bb2['y1'] < bb2['y2']

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1['x1'], bb2['x1'])
    y_top = max(bb1['y1'], bb2['y1'])
    x_right = min(bb1['x2'], bb2['x2'])
    y_bottom = min(bb1['y2'], bb2['y2'])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1['x2'] - bb1['x1']) * (bb1['y2'] - bb1['y1'])
    bb2_area = (bb2['x2'] - bb2['x1']) * (bb2['y2'] - bb2['y1'])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    assert iou >= 0.0
    assert iou <= 1.0
    return iou

In [25]:
appended_data = []
for i in tqdm(sorted(os.listdir(path))):
    df=pd.read_csv(path+i,delimiter=' ',header=None)
    df.columns=['class', 'x_centre', 'y_centre', 'width', 'height']
    df['vid']=int(i[:3])
    df['frame_id']=int(i[4:-4])
    df['bb_left']=df['x_centre']-df['width']/2.0
    df['bb_top']=df['y_centre']-df['height']/2.0
    df['confidence']=1
    df['class']=df['class']+1
    df = df[['vid','frame_id','bb_left','bb_top', 'width', 'height','class','confidence']]
    df=rm_overlap(df,0.5)
    #debug
    img=i[:-3]+'jpg'
    im = Image.open(img_path+img)
    width, height = im.size
    df['width']=round(df['width']*width).astype(int)
    df['bb_left']=round(df['bb_left']*width).astype(int)
    df['height']=round(df['height']*height).astype(int)
    df['bb_top']=round(df['bb_top']*height).astype(int)
    appended_data.append(df)
    
df    

100%|█████████████████████████████████████| 17188/17188 [03:39<00:00, 78.27it/s]


,vid,frame_id,bb_left,bb_top,width,height,class,confidence
0,100,99,993,54,23,50,2,1.0
1,100,99,303,673,73,46,5,1.0
2,100,99,664,520,71,154,5,1.0
3,100,99,328,647,91,71,2,1.0
4,100,99,556,247,51,89,2,1.0
5,100,99,419,248,53,97,2,1.0
6,100,99,546,275,94,76,1,1.0
7,100,99,419,313,49,61,1,1.0
8,100,99,665,531,111,176,2,1.0
9,100,99,656,632,124,86,1,1.0


In [30]:
#appended_data = pd.concat(appended_data)
appended_data=appended_data.sort_values(['vid', 'frame_id'], ascending=[True, True])
appended_data.head(50)

,vid,frame_id,bb_left,bb_top,width,height,class,confidence
0,1,1,348,230,35,65,2,1.0
1,1,1,348,230,35,66,3,1.0
2,1,1,239,283,33,51,1,1.0
3,1,1,239,247,32,66,2,1.0
4,1,1,349,263,35,53,1,1.0
0,1,2,352,233,36,68,2,1.0
1,1,2,242,289,32,47,1,1.0
2,1,2,241,250,33,65,2,1.0
3,1,2,353,264,36,58,1,1.0
0,1,3,357,238,38,71,3,1.0


In [32]:
appended_data.to_csv(out,index=False,header=None)